In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import qmc
from matplotlib import pyplot as plt

In [ ]:
ndim = 1

sobol = qmc.Sobol(ndim, scramble=False)
M = 1
N = 2**M
samp = sobol.random(N)
samp.shape

In [ ]:
pd.Series(np.ones_like(samp[:,0]),index=samp[:,0]).plot(style='.',ylim=(0.0,2.0),xlim=(0.0,1.0))

In [ ]:
ndim = 4

sobol = qmc.Sobol(ndim, scramble=True, seed=0)
M = 3
N = 2**M
samp = sobol.random(N)
samp.shape

In [ ]:
xdim = 0
ydim = 1

fig = plt.figure()
ax = fig.gca()
spacing = 1/8
minorLocator = plt.MultipleLocator(spacing)
ax.yaxis.set_minor_locator(minorLocator)
ax.xaxis.set_minor_locator(minorLocator)
ax.grid(which = 'minor')
ax.scatter(samp[:,xdim],samp[:,ydim])
ax.set_xlim(0.0,1.0)
ax.set_ylim(0.0,1.0)

In [ ]:
for i in range(32):
    lhs = qmc.LatinHypercube(ndim)
    print(qmc.discrepancy(lhs.random(N)))

In [ ]:
for i in range(32):
    sobol = qmc.Sobol(ndim, scramble=True)
    print(qmc.discrepancy(sobol.random(N)))

In [ ]:
from summer2 import CompartmentalModel, Stratification
from summer2.parameters import Parameter as param

In [ ]:
def sirs_parametric_age(times=[0, 100], agegroups=list(range(0, 80, 5))):
    m = CompartmentalModel(times, ["S", "I", "R"], "I")
    m.set_initial_population({"S": 99990.0, "I": 10.0})
    m.add_infection_frequency_flow("infection", param("contact_rate"), "S", "I")
    m.add_transition_flow("recovery", 1.0 / param("recovery_duration"), "I", "R")
    m.add_transition_flow("waning", 1.0 / param("waning_duration"), "R", "S")

    max_strl = len(str(agegroups[-1]))
    agegroup_keys = [str(k).zfill(max_strl) for k in agegroups]
    num_age = len(agegroup_keys)
    age_strat = Stratification("age", agegroup_keys, ["S", "I", "R"])

    # +++ SUMMER3
    # Not currently parameterizable
    suscept_adj = {}
    for k in agegroup_keys:
        suscept_adj[k] = np.exp(param(f"suscept_{k}"))
    age_strat.set_flow_adjustments("infection", suscept_adj)
    rec_adj = {k: adj for k, adj in zip(agegroup_keys, np.linspace(0.5, 2.5, num_age))}
    age_strat.set_flow_adjustments("recovery", rec_adj)
    wane_adj = {k: adj for k, adj in zip(agegroup_keys, np.linspace(2.5, 0.1, num_age))}
    age_strat.set_flow_adjustments("waning", wane_adj)

    #mm_base = np.linspace(5.5, 0.1, num_age).reshape((1, num_age))
    #mm = (mm_base * mm_base.T) * 0.1
    mm = np.random.uniform(size=(num_age,num_age))

    age_strat.set_mixing_matrix(mm)

    pop_spread = np.linspace(2.0, 1.0, num_age)
    pop_split = pop_spread / pop_spread.sum()

    age_strat.set_population_split({k: pop_prop for k, pop_prop in zip(agegroup_keys, pop_split)})

    m.stratify_with(age_strat)

    # +++ SUMMER3
    # Arbitrary dims for output requests
    # Export to xarray instead of dataframe
    incidence = m.request_output_for_flow("incidence", "infection")
    m.request_function_output("notifications", incidence * param("cdr"))

    for k in agegroup_keys:
        m.request_output_for_flow(f"incidenceXage_{k}", "infection", source_strata={"age": k})

    m.set_default_parameters(
        {"contact_rate": 0.04, "recovery_duration": 10.0, "waning_duration": 30.0, "cdr": 0.2} |
        {f"suscept_{k}":0.0 for k in agegroup_keys}
    )
    return m


In [ ]:
m = sirs_parametric_age()
agegroups = m.stratifications["age"].strata

In [ ]:
targetp = {f"suscept_{k}":np.random.normal(0.0,0.2) for k in agegroups}

In [ ]:
m.run(targetp)
inoisy = (m.get_derived_outputs_df()["incidence"] * np.exp(np.random.normal(0.0,0.1,101)))

In [ ]:
def pdopt(backend=None):
    if backend is None:
        if pd.options.plotting.backend == "plotly":
            backend = "matplotlib"
        else:
            backend = "plotly"
    pd.options.plotting.backend = backend

In [ ]:
pdopt("plotly")
m.get_derived_outputs_df().plot()

In [ ]:
pdopt()

In [ ]:
inoisy.plot()

In [ ]:
from estival import priors as esp
from estival import targets as est
from estival.model import BayesianCompartmentalModel

In [ ]:
defp = m.get_default_parameters()

In [ ]:
priors = [
    #esp.UniformPrior("contact_rate", (0.04,0.06)),
    #esp.UniformPrior("waning_duration", (15.0,25.0))
] + [
    esp.UniformPrior(f"suscept_{k}", (-1.0,1.0)) for k in agegroups
]
targets = [est.TruncatedNormalTarget("incidence",inoisy, (0.0,np.inf),inoisy*0.1)]
bcm = BayesianCompartmentalModel(m, defp, priors, targets)

In [ ]:
res = bcm.run(defp)

In [ ]:
def bcm_to_salib(bcm):
    problem = {
        "num_vars": len(bcm.priors),
        "names": [p for p in bcm.priors],
        "bounds": [p.bounds() for p in bcm.priors.values()]
    }
    return problem

In [ ]:
problem = bcm_to_salib(bcm)

In [ ]:
from SALib.analyze.sobol import analyze
from SALib.sample.sobol import sample

In [ ]:
from estival.sampling.tools import model_results_for_samples

In [ ]:
samp_df = pd.DataFrame(sample(problem, 32,scramble=True,seed=0,skip_values=0), columns=bcm.priors)
samp_df

In [ ]:
res = model_results_for_samples(samp_df, bcm)

In [ ]:
np.unique([k for k,_ in res.results.columns])

In [ ]:
y = ()

y = np.concatenate([y, res.results["incidence"].sum().to_numpy()])
ares = analyze(problem, y)


In [ ]:
df = ares.to_df()[0].sort_values("ST",ascending=False)
df

In [ ]:
sobol = qmc.Sobol(len(bcm.priors),scramble=True)

In [ ]:
samps = sobol.random(2048)

In [ ]:
sdf = bcm.sample.ppf(samps, "pandas")

In [ ]:
ssres = model_results_for_samples(sdf, bcm)

In [ ]:
sorted_res = sdf.loc[ssres.extras.sort_values("loglikelihood",ascending=False).index]
best_params = sdf.loc[sorted_res.index[0]]
sorted_res

In [ ]:
bcm.run(targetp).extras

In [ ]:
bcm.run(best_params).extras

In [ ]:
pd.DataFrame({
    "best": bcm.run(best_params).derived_outputs["incidence"],
    "target": bcm.run(targetp).derived_outputs["incidence"],
    "data": inoisy}).plot()